In [1]:
import sys

import numpy as np
import pandas as pd
from folktables import ACSDataSource, ACSIncome
import matplotlib.pyplot as plt

from lib.recursive_stopping import RecursiveStoppingRule
from lib.utils import load_ACS_income, preprocess_ACS_data, repair_year_state
# from lib.evals import eval_damage, eval_invariance

from pprint import pprint

In [2]:
data, labels = load_ACS_income(2018, 'CA')
data = preprocess_ACS_data(data, u='SCHL', u_thresh=16.0, s='RAC1P', s_thresh=1.0)

data

,AGEP,COW,MAR,OCCP,POBP,RELP,WKHP,SEX,u,s
0,30.0,6.0,1.0,9610.0,6.0,16.0,40.0,1,0,0
1,21.0,4.0,5.0,1970.0,6.0,17.0,20.0,1,1,1
2,65.0,2.0,5.0,2040.0,6.0,17.0,8.0,1,1,1
3,33.0,1.0,3.0,9610.0,36.0,16.0,40.0,1,0,1
4,18.0,2.0,5.0,1021.0,6.0,17.0,18.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...
195660,38.0,1.0,1.0,1021.0,210.0,0.0,40.0,1,1,0
195661,39.0,1.0,1.0,1021.0,210.0,1.0,40.0,0,1,0
195662,61.0,1.0,1.0,5240.0,17.0,0.0,45.0,1,1,1
195663,69.0,7.0,1.0,2040.0,207.0,0.0,45.0,1,1,0


In [3]:
N_bar = 5000
l = 0.1

features = data.drop(columns=['u','s']).columns

rules = {feat: {u: {s: RecursiveStoppingRule(N_bar=N_bar, x_min=-10000, x_max=10000, epsilon=1e-2, v_init=1e-4) for s in data['s'].unique()}
                for u in data['u'].unique()}
        for feat in features}

pprint(rules)

{'AGEP': {0: {0: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001),
              1: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001)},
          1: {0: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001),
              1: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001)}},
 'COW': {0: {0: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001),
             1: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001)},
         1: {0: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001),
             1: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001)}},
 'MAR': {0: {0: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001),
             1: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001)},
         1: {0: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001),
             1: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001)}},
 'OCCP': {0: {0: RecursiveStoppingRule(5000, -10000, 10000, 0.01, 0.0001),
              1: Recursi

In [ ]:
for i, row in data.iterrows():
        
    for f in features:
        # increment each stopping rule separately
        if rules[f][row['u']][row['s']].curr_rule.finished:
            continue
        else:
            rules[f][row['u']][row['s']].sample(row[f])

# rules[feat][u][s].increment_t(l)

In [ ]:
# test if all rules finished
for f in features:
    for u in data['u'].unique():
        for s in data['s'].unique():
            if not rules[f][row['u']][row['s']].curr_rule.finished:
                raise RuntimeWarning(f'{f}, ({u},{s}) did not finish.')

TO DO:
 - Conduct repair based on stopping numbers
 - Compile repaired on-sample and off-sample data
 - Produce eval report for the compiled data
 - PPrint eval report
 - Plot results
 - Group code into class
 - Test on 2014 - 2023 ACSIncome data.